# Conditional Linear Gaussian models

| | | |
|-|-|-|
|[ ![Creative Commons License](images/cc4.png)](http://creativecommons.org/licenses/by-nc/4.0/) |[ ![aGrUM](images/logoAgrum.png)](https://agrum.org) |[ ![interactive online version](images/atbinder.svg)](https://agrum.gitlab.io/extra/agrum_at_binder.html)

In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.bn_vs_bn as gcm

import pyAgrum.clg as gclg
import pyAgrum.clg.notebook as gclgnb


## Build a CLG model 

### From scratch

Suppose we want to build a CLG with these specifications $A={\cal N}(5,1)$, $B={\cal N}(4,3)$ and $C=2.A+3.B+{\cal N}(3,2)$

In [2]:
model=gclg.CLG()
model.add(gclg.GaussianVariable("A",5,1))
model.add(gclg.GaussianVariable("C",3,2))
model.add(gclg.GaussianVariable("B",4,3))
model.addArc("A","C",2)
model.addArc("B","C",3)
model

CLG{nodes: 3, arcs: 2, parameters: 8}

### From SEM (Structural Equation Model)

We can create a Conditional Linear Gaussian Bayesian networ(CLG model) using a SEM-like syntax. 

`A = 4.5 [0.3]` means that the mean of the distribution for Gaussian random variable A is 4.5 and ist standard deviation is 0.3. 

`B = 3 + 0.8F [0.3]` means that the mean of the distribution for the Gaussian random variable B is 3 and the standard deviation is 0.3.  

`pyAgrum.CLG.SEM` is a set of static methods to manipulate this kind of SEM.


In [3]:
sem2="""
A=4.5 [0.3] # comments are allowed
F=7 [0.5]
B=3 + 1.2F [0.3]
C=9 +  2A + 1.5B [0.6]
D=9 + C [0.7]
E=9 + D [0.9]
"""

model2 = gclg.SEM.toclg(sem2)

model2

CLG{nodes: 6, arcs: 5, parameters: 17}

One can of course build the SEM from a CLG using `pyAgrum.CLG.SEM.tosem`  :

In [4]:
gnb.flow.row(model,"<pre><div align='left'>"+gclg.SEM.tosem(model)+"</div></pre>",
             captions=["the first CLG model","the SEM from the CLG"])

And this SEM allows of course input/output format for CLG

In [5]:
gclg.SEM.saveCLG(model2,"out/model2.sem")

print("=== file content ===")
with open("out/model2.sem","r") as file:
  for line in file.readlines():
      print(line,end="")
print("====================")

=== file content ===
F=7.0[0.5]
B=3.0+1.2F[0.3]
A=4.5[0.3]
C=9.0+2.0A+1.5B[0.6]
D=9.0+C[0.7]
E=9.0+D[0.9]


In [6]:
model3=gclg.SEM.loadCLG("out/model2.sem")
gnb.sideBySide(model2,model3,captions=["saved model","loaded model"])

## Exact or approximated Inference

### Exact inference : Variable Elimination

Compute some posterior using difference exact inference

In [7]:
ie=gclg.CLGVariableElimination(model2)
ie.updateEvidence({"D":3})

print(ie.posterior("A"))
print(ie.posterior("B"))
print(ie.posterior("C"))
print(ie.posterior("D"))
print(ie.posterior("E"))
print(ie.posterior("F"))


v=ie.posterior("E")
print(v)
print(f"  - mean(E|D=3)={v.mu()}")
print(f"  - stdev(E|D=3)={v.sigma()}")

A:1.1713160854893137[0.2746303374942688]
B:-1.082564679415073[0.4949690654000884]
C:3.0614173228346453[0.6180360053726707]
D:3[0]
E:12.0[0.9]
F:-1.3217097862767153[0.3986055559087828]
E:12.0[0.9]
  - mean(E|D=3)=12.0
  - stdev(E|D=3)=0.9


In [8]:
gnb.sideBySide(model2,gclgnb.getInference(model2,evs={"D":3},size="3!"),gclgnb.getInference(model2,evs={"D":3,"F":1}),
              captions=["The CLG","First inference","Second inference"])

### Approximated inference : MonteCarlo Sampling

When the model is too complex for exact infernece, we can use forward sampling to generate 5000 samples from the original CLG model. 

In [9]:
fs = gclg.ForwardSampling(model2)
fs.makeSample(5000).tocsv("./out/model2.csv")

We will use the generated database to do learning. But before, we can also compute posterior but without evidence :

In [10]:
ie=gclg.CLGVariableElimination(model2)
print("| 'Exact' inference                        | Results from sampling                    |")
print("|------------------------------------------|------------------------------------------|")
for i in model2.names():
    print(f"| {str(ie.posterior(i)):40} | {str(gclg.GaussianVariable(i,fs.mean_sample(i),fs.stddev_sample(i))):40} |")

| 'Exact' inference                        | Results from sampling                    |
|------------------------------------------|------------------------------------------|
| A:4.5[0.3]                               | A:4.500275244120617[0.29679150013427535] |
| F:6.999999999999992[0.4999999999999998]  | F:6.988369942053467[0.4926974896851237]  |
| B:11.399999999999984[0.6708203932499365] | B:11.378099940112547[0.66552384421092]   |
| C:35.09999999999995[1.3162446581088174]  | C:35.06346745381681[1.3054360733448969]  |
| D:44.09999999999998[1.490805151587557]   | D:44.081827105799[1.4681230379376173]    |
| E:53.099999999999945[1.741407476726799]  | E:53.05797237363066[1.7455343500113596]  |


Now with the generated database and the original model, we can calculate the log-likelihood of the model.

In [11]:
print("log-likelihood w.r.t orignal model : ", model2.logLikelihood("./out/model2.csv"))

log-likelihood w.r.t orignal model :  -22287.55081936626


## Learning a CLG

Use the generated database to do our RAvel Learning. This part needs some time to run.

In [12]:
# RAveL learning
learner = gclg.RAveL_learning("./out/model2.csv")

We can get the learned_clg model with function learn_clg() which contains structure learning and parameter estimation.

In [13]:
learned_clg = learner.learnCLG()
gnb.sideBySide(model2,learned_clg,
              captions=['original CLG','learned CLG'])

Compare the learned model's structure with that of the original model'.

In [14]:
cmp=gcm.GraphicalBNComparator(model2,learned_clg)
print(f"F-score(original_clg,learned_clg) : {cmp.scores()['fscore']}")

F-score(original_clg,learned_clg) : 1.0


Get the learned model's parameters and compare them with the original model's parameters using the SEM syntax.

In [15]:
gnb.flow.row("<pre><div align='left'>"+gclg.SEM.tosem(model2)+"</div></pre>",
             "<pre><div align='left'>"+gclg.SEM.tosem(learned_clg)+"</div></pre>",
             captions=["original sem","learned sem"])

We can algo do parameter estimation only with function fitParameters() if we already have the structure of the model.

In [16]:
# We can copy the original CLG
copy_original = gclg.CLG(model2)

# RAveL learning again
RAveL_l = gclg.RAveL_learning("./out/model2.csv")

# Fit the parameters of the copy clg
RAveL_l.fitParameters(copy_original)

copy_original

CLG{nodes: 6, arcs: 5, parameters: 17}

## Compare two CLG models

We first create two CLG from two SEMs.

In [17]:
# TWO DIFFERENT CLGs

# FIRST CLG
clg1=gclg.SEM.toclg("""
# hyper parameters
A=4[1]
B=3[5]
C=-2[5]

#equations
D=A[.2] # D is a noisy version of A
E=1+D+2B [2]
F=E+C+B+E [0.001]
""")

# SECOND CLG
clg2=gclg.SEM.toclg("""
# hyper parameters
A=4[1]
B=3+A[5]
C=-2+2B+A[5]

#equations
D=A[.2] # D is a noisy version of A
E=1+D+2B [2]
F=E+C [0.001]
""")

This cell shows how to have a quick view of the differences 

In [18]:
gnb.flow.row(clg1,clg2,gcm.graphDiff(clg1,clg2),gcm.graphDiffLegend(),gcm.graphDiff(clg2,clg1))

We compare the CLG models.

In [19]:
# We use the F-score to compare the two CLGs
cmp=gcm.GraphicalBNComparator(clg1,clg1)
print(f"F-score(clg1,clg1) : {cmp.scores()['fscore']}")

cmp=gcm.GraphicalBNComparator(clg1,clg2)
print(f"F-score(clg1,clg2) : {cmp.scores()['fscore']}")


F-score(clg1,clg1) : 1.0
F-score(clg1,clg2) : 0.7142857142857143


In [20]:
# The complete list of structural scores is :
print("score(clg1,clg2) :")
for score,val in cmp.scores().items():
  print(f"  - {score} : {val}")

score(clg1,clg2) :
  - count : {'tp': 5, 'tn': 21, 'fp': 3, 'fn': 1}
  - recall : 0.8333333333333334
  - precision : 0.625
  - fscore : 0.7142857142857143
  - dist2opt : 0.41036907507483766


## Forward Sampling

In [21]:
# We create a simple CLG with 3 variables
clg = gclg.CLG()
# prog=« sigma=2;X=N(5);Y=N(3);Z=X+Y »
A = gclg.GaussianVariable(mu=2, sigma=1, name='A')
B = gclg.GaussianVariable(mu=1, sigma=2, name='B')
C = gclg.GaussianVariable(mu=2, sigma=3, name='C')
  
idA = clg.add(A)
idB = clg.add(B)
idC = clg.add(C)

clg.addArc(idA, idB, 1.5)
clg.addArc(idB, idC, 0.75)

# We can show it as a graph
original_clg = gclgnb.CLG2dot(clg)
original_clg

In [22]:
fs = gclg.ForwardSampling(clg)
fs.makeSample(10)

In [23]:
print("A's sample_variance: ", fs.variance_sample(0))
print("B's sample_variance: ", fs.variance_sample('B'))
print("C's sample_variance: ", fs.variance_sample(2))

A's sample_variance:  0.7206043646910405
B's sample_variance:  5.250752105987472
C's sample_variance:  6.63105555376981


In [24]:
print("A's sample_mean: ", fs.mean_sample('A'))
print("B's sample_mean: ", fs.mean_sample('B'))
print("C's sample_mean: ", fs.mean_sample('C'))

A's sample_mean:  2.289227013923856
B's sample_mean:  4.761965490406184
C's sample_mean:  4.799459458386599


In [25]:
fs.toarray()

array([[1.86985212, 7.24686302, 3.54587413],
       [1.65516186, 3.16668487, 1.41368752],
       [3.2821574 , 5.44144195, 7.87372707],
       [3.615807  , 6.06861955, 7.80063318],
       [0.58444318, 0.22888315, 3.4380197 ],
       [2.25442291, 7.04499903, 6.26226083],
       [2.86578088, 6.60875788, 8.7696211 ],
       [2.30387136, 3.32408329, 1.66980397],
       [2.7940143 , 6.46326642, 4.71499752],
       [1.66675913, 2.02605575, 2.50596957]])

In [26]:
# export to dataframe
fs.topandas()

,A,B,C
0,1.869852,7.246863,3.545874
1,1.655162,3.166685,1.413688
2,3.282157,5.441442,7.873727
3,3.615807,6.068620,7.800633
4,0.584443,0.228883,3.438020
5,2.254423,7.044999,6.262261
6,2.865781,6.608758,8.769621
7,2.303871,3.324083,1.669804
8,2.794014,6.463266,4.714998
9,1.666759,2.026056,2.505970


In [27]:
# export to csv
fs.makeSample(10000)
fs.tocsv('./out/samples.csv')

## PC-algorithm & Parameter Estimation

The module allows to investigale more deeply into the learning algorithm.

We first create a random CLG model with 5 variables.

In [28]:
# Create a new random CLG
clg = gclg.CLG.randomCLG(nb_variables=5, names="ABCDE")

# Display the CLG
clg

CLG{nodes: 5, arcs: 5, parameters: 15}

We then do the Forward Sampling and RAveL_learning.

In [29]:
n = 20 # n is the selected values of MC number n in n-MCERA
K = 10000 # K is the list of selected values of number of samples
Delta = 0.05 # Delta is the FWER we want to control

# Sample generation
fs = gclg.ForwardSampling(clg)
fs.makeSample(K).tocsv('./out/clg.csv')

# Learning
RAveL_l = gclg.RAveL_learning('./out/clg.csv',n_sample=n,fwer_delta=Delta)

We use the PC algorithme to learn the structure of the model.

In [30]:
# Use the PC algorithm to get the skeleton
C = RAveL_l.PC_algorithm(order=clg.nodes(), verbose=False)
print("The final skeleton is:\n", C)

The final skeleton is:
 {0: {1}, 1: {4}, 2: {3, 4}, 3: set(), 4: {3}}


In [31]:
# Create a Mixedgraph to display the skeleton
RAveL_MixGraph = gum.MixedGraph()

# Add variables
for i in range(len(clg.names())):
  RAveL_MixGraph.addNodeWithId(i)

# Add arcs and edges
for father, kids in C.items():
  for kid in kids:
    if father in C[kid]:
      RAveL_MixGraph.addEdge(father, kid)
    else:
      RAveL_MixGraph.addArc(father, kid)

RAveL_MixGraph

(pyAgrum.MixedGraph@000002082D206890) {0,1,2,3,4} , {0->1,2->3,4->3,1->4,2->4} , {}

In [32]:
# Create a BN with the same structure as the CLG
bn = gum.BayesNet()
# add variables
for name in clg.names():
  new_variable = gum.LabelizedVariable(name,'a labelized variable',2)
  bn.add(new_variable)
# add arcs
for arc in clg.arcs():
  bn.addArc(arc[0], arc[1])

# Compare the result above with the EssentialGraph
Real_EssentialGraph = gum.EssentialGraph(bn)

Real_EssentialGraph

<pyAgrum.pyAgrum.EssentialGraph; proxy of <Swig Object of type 'gum::EssentialGraph *' at 0x0000020830EF81B0> >

In [33]:
# create a CLG from the skeleton of PC algorithm
clg_PC = gclg.CLG()
for node in clg.nodes():
  clg_PC.add(clg.variable(node))
for father,kids in C.items():
  for kid in kids:
    clg_PC.addArc(father, kid)

# Compare the structure of the created CLG and the original CLG
print(f"F-score : {clg.CompareStructure(clg_PC)}")

F-score : 0.8000000000000002


We can also do the parameter learning.

In [34]:
id2mu, id2sigma, arc2coef = RAveL_l.estimate_parameters(C)

for node in clg.nodes():
  print(f"Real Value: node {node} : mu = {clg.variable(node)._mu}, sigma = {clg.variable(node)._sigma}")
  print(f"Estimation: node {node} : mu = {id2mu[node]}, sigma = {id2sigma[node]}")


for arc in clg.arcs():
  print(f"Real Value: arc {arc} : coef = {clg.coefArc(*arc)}")
  print(f"Estimation: arc {arc} : coef = {(arc2coef[arc] if arc in arc2coef else '-')}")

Real Value: node 0 : mu = 2.493731938052063, sigma = 9.498135821313946
Estimation: node 0 : mu = 2.452683017219798, sigma = 9.47666972468408
Real Value: node 1 : mu = -0.12493299028560045, sigma = 9.501670507500059
Estimation: node 1 : mu = -0.2966117895935483, sigma = 9.49724686517989
Real Value: node 2 : mu = 2.3925582788837643, sigma = 8.274163841373582
Estimation: node 2 : mu = 2.4504625869723835, sigma = 8.343255265758184
Real Value: node 3 : mu = -4.461786795326966, sigma = 6.791001448641218
Estimation: node 3 : mu = -6.064456165440333, sigma = 12.344732067641495
Real Value: node 4 : mu = 4.56810109093033, sigma = 3.821339746517472
Estimation: node 4 : mu = 4.607810683009731, sigma = 3.784187462016438
Real Value: arc (0, 1) : coef = 5.207810430437469
Estimation: arc (0, 1) : coef = 5.206925357461134
Real Value: arc (2, 4) : coef = -7.40053273188409
Estimation: arc (2, 4) : coef = -7.406935857108638
Real Value: arc (4, 3) : coef = -9.806308989982263
Estimation: arc (4, 3) : coef =